In [1]:
%load_ext autoreload
%autoreload 2
import os
import torch as t
from utils.config import opt
from model import FasterRCNNVGG16
from trainer import FasterRCNNTrainer
from data.util import  read_image
from utils.vis_tool import vis_bbox
from utils import array_tool as at
%matplotlib inline
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from data.dataset import Dataset, TestDataset, inverse_normalize
from utils.config import opt
from utils.eval_tool import eval_detection_voc
import sys
from torch.utils import data as data_
from tqdm import tqdm



In [2]:
testset = TestDataset(opt, scale=0.5)
test_dataloader = data_.DataLoader(testset,
                                   batch_size=1,
                                   num_workers=1,
                                   shuffle=False, \
                                   pin_memory=True
                                   )
img, sizes, gt_bboxes_, gt_labels_, gt_difficults_ = next(iter(test_dataloader))


In [3]:
f, axarr = plt.subplots(1,1)
axarr.imshow(img[0,0,:,:], cmap='gray')
axarr.axis('off')
# gt_boxes = extract_boxes(path.replace('.jpg', '.txt'))

for box in gt_bboxes_[0]:
    rect = patches.Rectangle((box[1],box[0]),box[3]-box[1],box[2]-box[0],linewidth=2,edgecolor='g',facecolor='none')
    axarr.add_patch(rect)


axarr.title.set_text('Ground Truth')


In [4]:
def print_inline(line):
    sys.stdout.write(f'\r{line}')
    sys.stdout.flush()

In [5]:
faster_rcnn = FasterRCNNVGG16()
opt.caffe_pretrain=False 
trainer = FasterRCNNTrainer(faster_rcnn).cuda()
trainer.load('checkpoints/ckpt.pt')


Traceback (most recent call last):
  File "/home/nasir/anaconda3/envs/sar/lib/python3.6/site-packages/urllib3/connection.py", line 159, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/home/nasir/anaconda3/envs/sar/lib/python3.6/site-packages/urllib3/util/connection.py", line 80, in create_connection
    raise err
  File "/home/nasir/anaconda3/envs/sar/lib/python3.6/site-packages/urllib3/util/connection.py", line 70, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/nasir/anaconda3/envs/sar/lib/python3.6/site-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/home/nasir/anaconda3/envs/sar/lib/python3.6/site-packages/urllib3/connectionpool.py", line 354, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/home/nasir/anaconda3/envs

Exception in user code:
------------------------------------------------------------


FasterRCNNTrainer(
  (faster_rcnn): FasterRCNNVGG16(
    (extractor): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace)
      (16): MaxPoo

In [9]:
import glob
paths = glob.glob('images/*.jpg')

In [10]:
def str2int(a):
    return [int(x) for x in a]

def extract_boxes(fname):
    with open(fname) as f:
        content = f.readlines()
        f.close()
        content = [x.strip() for x in content]
        content = [str2int(x.split(' ')[-4:]) for x in content]
        return content

In [11]:
for path in paths:
    img = read_image(path)
    img = t.from_numpy(img)[None]
    _bboxes, _labels, _scores = trainer.faster_rcnn.predict(img,visualize=True)

    f, axarr = plt.subplots(1,2)
    axarr[0].imshow(img[0,0,:,:], cmap='gray')
    axarr[0].axis('off')
    axarr[1].imshow(img[0,0,:,:], cmap='gray')
    axarr[1].axis('off')
    gt_boxes = extract_boxes(path.replace('.jpg', '.txt'))

    for box in gt_boxes:
        rect = patches.Rectangle((box[0],box[1]),box[2],box[3],linewidth=2,edgecolor='g',facecolor='none')
        axarr[0].add_patch(rect)

    for box in _bboxes[0]:
        rect = patches.Rectangle((box[1],box[0]),box[3]-box[1],box[2]-box[0],linewidth=2,edgecolor='r',facecolor='none')
        axarr[1].add_patch(rect)

    axarr[0].title.set_text('Ground Truth')
    axarr[1].title.set_text('Detected')
    plt.savefig(path.replace('images', 'results'))

In [12]:
def eval(dataloader, faster_rcnn, test_num=10000):
    pred_bboxes, pred_labels, pred_scores = list(), list(), list()
    gt_bboxes, gt_labels, gt_difficults = list(), list(), list()
    for ii, (imgs, sizes, gt_bboxes_, gt_labels_, gt_difficults_) in tqdm(enumerate(dataloader)):
        sizes = [sizes[0][0].item(), sizes[1][0].item()]
        pred_bboxes_, pred_labels_, pred_scores_ = faster_rcnn.predict(imgs, [sizes])
        gt_bboxes += list(gt_bboxes_.numpy())
        gt_labels += list(gt_labels_.numpy())
        gt_difficults += list(gt_difficults_.numpy())
        pred_bboxes += pred_bboxes_
        pred_labels += pred_labels_
        pred_scores += pred_scores_
        if ii == test_num: break
    result = eval_detection_voc(
        pred_bboxes, pred_labels, pred_scores,
        gt_bboxes, gt_labels, gt_difficults,
        use_07_metric=True)
    return result

In [13]:
testset = TestDataset(opt, scale=0.6)
test_dataloader = data_.DataLoader(testset,
                                   batch_size=1,
                                   num_workers=1,
                                   shuffle=False, \
                                   pin_memory=True
                                   )
eval_result = eval(test_dataloader, faster_rcnn, test_num=opt.test_num)
print('test_map', eval_result['map'])

3810it [06:36,  9.23it/s]


test_map 0.4166509698414168


In [11]:
testset = TestDataset(opt, scale=0.85)
test_dataloader = data_.DataLoader(testset,
                                   batch_size=1,
                                   num_workers=1,
                                   shuffle=False, \
                                   pin_memory=True
                                   )
eval_result = eval(test_dataloader, faster_rcnn, test_num=opt.test_num)
print('test_map', eval_result['map'])

3810it [09:31,  8.26it/s]


test_map 0.636499810749856


In [12]:
testset = TestDataset(opt, scale=1)
test_dataloader = data_.DataLoader(testset,
                                   batch_size=1,
                                   num_workers=1,
                                   shuffle=False, \
                                   pin_memory=True
                                   )
eval_result = eval(test_dataloader, faster_rcnn, test_num=opt.test_num)
print('test_map', eval_result['map'])

3810it [09:09,  6.88it/s]


test_map 0.7574635787177146


In [13]:
testset = TestDataset(opt, scale=1.1)
test_dataloader = data_.DataLoader(testset,
                                   batch_size=1,
                                   num_workers=1,
                                   shuffle=False, \
                                   pin_memory=True
                                   )
eval_result = eval(test_dataloader, faster_rcnn, test_num=opt.test_num)
print('test_map', eval_result['map'])

3810it [09:26,  7.78it/s]


test_map 0.7356694260122546


In [14]:
testset = TestDataset(opt, scale=1.23)
test_dataloader = data_.DataLoader(testset,
                                   batch_size=1,
                                   num_workers=1,
                                   shuffle=False, \
                                   pin_memory=True
                                   )
eval_result = eval(test_dataloader, faster_rcnn, test_num=opt.test_num)
print('test_map', eval_result['map'])

3810it [09:43,  7.85it/s]


test_map 0.7222116736644185


In [15]:
testset = TestDataset(opt, scale=1.25)
test_dataloader = data_.DataLoader(testset,
                                   batch_size=1,
                                   num_workers=1,
                                   shuffle=False, \
                                   pin_memory=True
                                   )
eval_result = eval(test_dataloader, faster_rcnn, test_num=opt.test_num)
print('test_map', eval_result['map'])

3810it [09:21,  7.66it/s]


test_map 0.7456952756721609


In [16]:
testset = TestDataset(opt, scale=1.35)
test_dataloader = data_.DataLoader(testset,
                                   batch_size=1,
                                   num_workers=1,
                                   shuffle=False, \
                                   pin_memory=True
                                   )
eval_result = eval(test_dataloader, faster_rcnn, test_num=opt.test_num)
print('test_map', eval_result['map'])

3810it [09:59,  7.14it/s]


test_map 0.7023230430179115


In [17]:
testset = TestDataset(opt, scale=1.48)
test_dataloader = data_.DataLoader(testset,
                                   batch_size=1,
                                   num_workers=1,
                                   shuffle=False, \
                                   pin_memory=True
                                   )
eval_result = eval(test_dataloader, faster_rcnn, test_num=opt.test_num)
print('test_map', eval_result['map'])

3810it [09:53,  7.13it/s]


test_map 0.6495383028342389


In [18]:
testset = TestDataset(opt, scale=2.0)
test_dataloader = data_.DataLoader(testset,
                                   batch_size=1,
                                   num_workers=1,
                                   shuffle=False, \
                                   pin_memory=True
                                   )
eval_result = eval(test_dataloader, faster_rcnn, test_num=opt.test_num)
print('test_map', eval_result['map'])

3810it [10:29,  6.06it/s]


test_map 0.22073866376076506
